In [ ]:
import tensorflow
import pickle
import numpy as np
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import pickle

In [ ]:
file = open('Data/sentiment_set.pickle','rb')
train_X,train_Y,test_X,test_Y = pickle.load(file)

In [ ]:
# features

In [ ]:
train[1:]

In [ ]:
# number of nodes in each layer
n_nodes_hl1 = 700
n_nodes_hl2 = 600
n_nodes_hl3 = 600

n_classes = 2
batch_size = 500
# [height,width]
x = tf.placeholder('float')
# if we do not feed data of this shape to x it'll show error
y = tf.placeholder('float')

def neural_network_model(data):
    # just defining weigths and baises
    # weights inputs shape , nodes in layers
    # bais are always one.
    # the bais are kept because suppose if all the features are 0 then w*features =0 so no neuron activate. hence bais keeps some neurons activated
    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([len(train_X[0]), n_nodes_hl1])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}
    
    # now inoput will be depending on the shappe of previous layers
    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}

    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}

    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                    'biases':tf.Variable(tf.random_normal([n_classes]))}
    
    # calculations
    l1 = tf.add(tf.matmul(data,hidden_1_layer['weights']), hidden_1_layer['biases'])
    # activation function
    l1 = tf.nn.relu(l1)

    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)

    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)

    output = tf.matmul(l3,output_layer['weights']) + output_layer['biases']

    return output

def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
    # learning rate by default here is 0.001
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    hm_epochs=10
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(hm_epochs):
            epoch_loss = 0
            # if we do not use the variable we're using fore iteration there we use '_' as variables.
            i=0
            while i < len(train_X):
                start = i
                end = i + batch_size
                batch_x = np.array(train_X[start:end])
                batch_y = np.array(train_Y[start:end])
                # here we try to optimize the cost 
                _, c = sess.run([optimizer, cost], feed_dict={x: batch_x, y: np.array(batch_y[:,0]).reshape(-1,1)})
                epoch_loss += c
                i+=batch_size


            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)
            plt.scatter(epoch,epoch_loss)
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:test_X, y:test_Y}))
        plt.show()
train_neural_network(x)